<a href="https://colab.research.google.com/github/JKDS87/Sales-Predictions/blob/main/Sales_Predictions_NB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prediction of Sales

Student: Matthew Malueg

# Load libraries and files

**Loading and Importing**

In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import packages for column transformation
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import missingno
pd.set_option('display.max_columns',100)
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
# Set pandas as the default output for sklearn
from sklearn import set_config
set_config(transform_output='pandas')

In [ ]:
# Load in the data from Google Drive, set the index, and preview first 5 rows
fpath = '/content/drive/MyDrive/CodingDojo/01-Fundamentals/Week02/Data/sales_predictions_2023.csv'
df = pd.read_csv(fpath)

**Custom Functions**

In [ ]:
def regression_metrics(y_true, y_pred, label='', verbose = True, output_dict=False):
  # Get metrics
  mae = mean_absolute_error(y_true, y_pred)
  mse = mean_squared_error(y_true, y_pred)
  rmse = mean_squared_error(y_true, y_pred, squared=False)
  r_squared = r2_score(y_true, y_pred)
  if verbose == True:
    # Print Result with Label and Header
    header = "-"*60
    print(header, f"Regression Metrics: {label}", header, sep='\n')
    print(f"- MAE = {mae:,.3f}")
    print(f"- MSE = {mse:,.3f}")
    print(f"- RMSE = {rmse:,.3f}")
    print(f"- R^2 = {r_squared:,.3f}")
  if output_dict == True:
      metrics = {'Label':label, 'MAE':mae,
                 'MSE':mse, 'RMSE':rmse, 'R^2':r_squared}
      return metrics

def evaluate_regression(reg, X_train, y_train, X_test, y_test, verbose = True,
                        output_frame=False):
  # Get predictions for training data
  y_train_pred = reg.predict(X_train)

  # Call the helper function to obtain regression metrics for training data
  results_train = regression_metrics(y_train, y_train_pred, verbose = verbose,
                                     output_dict=output_frame,
                                     label='Training Data')
  print()
  # Get predictions for test data
  y_test_pred = reg.predict(X_test)
  # Call the helper function to obtain regression metrics for test data
  results_test = regression_metrics(y_test, y_test_pred, verbose = verbose,
                                  output_dict=output_frame,
                                    label='Test Data' )

  # Store results in a dataframe if ouput_frame is True
  if output_frame:
    results_df = pd.DataFrame([results_train,results_test])
    # Set the label as the index
    results_df = results_df.set_index('Label')
    # Set index.name to none to get a cleaner looking result
    results_df.index.name=None
    # Return the dataframe
    return results_df.round(3)

# Initial inspection and cleaning

## Inspect data and drop duplicates

In [ ]:
# Examine data for errors, or inconsistency in labeling and categorical data
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [ ]:
df.nunique() / len(df) * 100

Item_Identifier              18.291681
Item_Weight                   4.869178
Item_Fat_Content              0.058665
Item_Visibility              92.455708
Item_Type                     0.187727
Item_MRP                     69.670304
Outlet_Identifier             0.117330
Outlet_Establishment_Year     0.105597
Outlet_Size                   0.035199
Outlet_Location_Type          0.035199
Outlet_Type                   0.046932
Item_Outlet_Sales            40.983222
dtype: float64

In [ ]:
# Check for any duplicates
df.duplicated().sum()

0

In [ ]:
# Drop Item_Identifier and verify
df = df.drop(columns='Item_Identifier')
df.head()

## Correcting strings

In [ ]:
# Inspect 'Outlet_Identifier'
df['Outlet_Identifier'].value_counts()

# The string 'OUT' is extraneous.

OUT027    935
OUT013    932
OUT049    930
OUT046    930
OUT035    930
OUT045    929
OUT018    928
OUT017    926
OUT010    555
OUT019    528
Name: Outlet_Identifier, dtype: int64

In [ ]:
# Remove string characters and convert to int
df['Outlet_Identifier'] = df['Outlet_Identifier'].str.replace('OUT', '')#.astype('int')
df['Outlet_Identifier'] = df['Outlet_Identifier'].astype('int')
df['Outlet_Identifier'].value_counts()

27    935
13    932
49    930
46    930
35    930
45    929
18    928
17    926
10    555
19    528
Name: Outlet_Identifier, dtype: int64

In [ ]:
# Inspect 'Outlet_Location_Type'
df['Outlet_Location_Type'].value_counts()

# The string 'Tier ' is extraneous.

Tier 3    3350
Tier 2    2785
Tier 1    2388
Name: Outlet_Location_Type, dtype: int64

In [ ]:
# Remove string characters and convert to int
df['Outlet_Location_Type'] = df['Outlet_Location_Type'].str.replace('Tier ', '')
df['Outlet_Location_Type'] = df['Outlet_Location_Type'].astype('int')
df['Outlet_Location_Type'].value_counts()

3    3350
2    2785
1    2388
Name: Outlet_Location_Type, dtype: int64

In [ ]:
# Inspect 'Item_Fat_Content' for inconsistencies
df['Item_Fat_Content'].value_counts()

# We can consolidate some of these values.

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [ ]:
# Replace strings to consolidate values
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace({'LF': 'Low Fat',
                                                         'low fat': 'Low Fat',
                                                         'reg': 'Regular'})
# Inspect 'Item_Fat_Content' to verify change
df['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

In [ ]:
# Check amount of missing values
df.isna().sum()

Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [ ]:
# Inspect 'Outlet_Size' for inconsistencies
df['Outlet_Size'].value_counts()

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64

In [ ]:
# Inspect 'Outlet_Type' for inconsistencies
df['Outlet_Type'].value_counts()

Supermarket Type1    5577
Grocery Store        1083
Supermarket Type3     935
Supermarket Type2     928
Name: Outlet_Type, dtype: int64

In [ ]:
# Check unique values
df.nunique()

Item_Weight                   415
Item_Fat_Content                2
Item_Visibility              7880
Item_Type                      16
Item_MRP                     5938
Outlet_Identifier              10
Outlet_Establishment_Year       9
Outlet_Size                     3
Outlet_Location_Type            3
Outlet_Type                     4
Item_Outlet_Sales            3493
dtype: int64

Although 'Item_Visibility' and 'Item_MRP' have high cardinality, they are a percentage and price respectively, and not categorical data. As such having a large variety of exact figures for floats makes sense.

All categorical data seems to be in order, feature types match data, and duplicates have been removed. Move on to preprocessing.

# Preprocessing ColumnTransformer

**Train Test Split**

In [ ]:
# Split our database into the target vector y and feature matrix X
# Set target
y = df['Item_Outlet_Sales']
# Set feature matrix
X = df.drop(columns='Item_Outlet_Sales')

In [ ]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

Use the ColumnTransformer created below to clean and fit model to training data.

**Numeric pipeline**

In [ ]:
# List of numeric column names
num_cols = X_train.select_dtypes('number').columns

# Instantiate preprocessors for numeric cols
impute_median = SimpleImputer(strategy='median')
scaler = StandardScaler()

# Instantiate numeric preprocessing pipeline
num_pipe = make_pipeline(impute_median, scaler)

# Create numeric touple for ColumnTransformer
num_tuple = ('numeric', num_pipe, num_cols)

**Ordinal pipeline**

In [ ]:
# List of ordinal features
ord_cols = ['Outlet_Size']

# Instantiate SimpleImputer
impute_na_ord = SimpleImputer(strategy='constant', fill_value='NA')
## Create the OrdinalEncoder
# Specify order of categories in ordinal columns
outlet_size_order = ['NA', 'Small', 'Medium', 'High']
# Make the list of ordered lists for OrdinalEncoder
ordinal_cat_orders = [outlet_size_order]
ord_encoder = OrdinalEncoder(categories=ordinal_cat_orders)
# Instantiate StandardScaler for ordinal cols
scaler_ord = StandardScaler()

# Instantiate ordinal preprocessing pipeline
ord_pipe = make_pipeline(impute_na_ord, ord_encoder, scaler_ord)

# Create ordinal tuple for ColumnTransformer
ord_tuple = ('ordinal', ord_pipe, ord_cols)

**Categorical pipeline**

In [ ]:
# List of categorical features
ohe_cols = X_train.select_dtypes('object').drop(columns=ord_cols).columns

# Instantiate the preprocessors
impute_na = SimpleImputer(strategy='constant', fill_value = 'NA')
ohe_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Instantiate the pipeline
ohe_pipe = make_pipeline(impute_na, ohe_encoder)

# Create categorical tuple for ColumnTransfer
ohe_tuple = ('categorical', ohe_pipe, ohe_cols)

**Create ColumnTransformer using preprocessor pipelines**

In [ ]:
# Create the column transformer by using the three pipelines
preprocessor = ColumnTransformer([num_tuple, ord_tuple, ohe_tuple],
                                    verbose_feature_names_out=False)
preprocessor

ColumnTransformer(transformers=[('numeric',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 Index(['Item_Weight', 'Item_Visibility', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Location_Type'],
      dtype='object')),
                                ('ordinal',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_...
                                                                              'Medium',
                                                                              'High']])),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['Outlet_Size']),
                                ('categorical',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='NA',
                                                                strategy='constant')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 Index(['Item_Fat_Content', 'Item_Type', 'Outlet_Type'], dtype='object'))],
                  verbose_feature_names_out=False)

# Modeling

## Linear Regression

In [ ]:
# Create a linear regression pipeline
linreg = LinearRegression()
# Combine preprocessing and linreg model in one pipeline
linreg_pipe = make_pipeline(preprocessor, linreg)
# Fitting model pipeline on training data
linreg_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  Index(['Item_Weight', 'Item_Visibility', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Location_Type'],
      dtype='object')),
                                                 ('ordinal',
                                                  Pipeline(steps=...
                                                                   StandardScaler())]),
                                                  ['Outlet_Size']),
                                                 ('categorical',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='NA',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  Index(['Item_Fat_Content', 'Item_Type', 'Outlet_Type'], dtype='object'))],
                                   verbose_feature_names_out=False)),
                ('linearregression', LinearRegression())])

In [ ]:
# Obtain model evaluation using saved custom functions
evaluate_regression(linreg_pipe, X_train, y_train, X_test, y_test)

------------------------------------------------------------
Regression Metrics: Training Data
------------------------------------------------------------
- MAE = 847.474
- MSE = 1,297,939.721
- RMSE = 1,139.272
- R^2 = 0.561

------------------------------------------------------------
Regression Metrics: Test Data
------------------------------------------------------------
- MAE = 803.859
- MSE = 1,194,511.182
- RMSE = 1,092.937
- R^2 = 0.567


This model currently has R2 scores of 0.561 and 0.567 for training and testing data. With both scores being rather inaccurate, we can surmize this model is underfit with a high bias.

## Random Forest

**Default Random Forest Model**

In [ ]:
# Instantiate a default Random Forest model
rf = RandomForestRegressor()
# Model pipeline with preprocessor and default rf model
rf_pipe = make_pipeline(preprocessor, rf)
# Fit model pipeline on training data only
rf_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  Index(['Item_Weight', 'Item_Visibility', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Location_Type'],
      dtype='object')),
                                                 ('ordinal',
                                                  Pipeline(steps=...
                                                  ['Outlet_Size']),
                                                 ('categorical',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='NA',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  Index(['Item_Fat_Content', 'Item_Type', 'Outlet_Type'], dtype='object'))],
                                   verbose_feature_names_out=False)),
                ('randomforestregressor', RandomForestRegressor())])

In [ ]:
# Obtain model evaluation using saved custom functions
evaluate_regression(rf_pipe, X_train, y_train, X_test, y_test)

------------------------------------------------------------
Regression Metrics: Training Data
------------------------------------------------------------
- MAE = 297.019
- MSE = 183,412.209
- RMSE = 428.267
- R^2 = 0.938

------------------------------------------------------------
Regression Metrics: Test Data
------------------------------------------------------------
- MAE = 777.504
- MSE = 1,247,129.917
- RMSE = 1,116.750
- R^2 = 0.548


This model currently has R2 scores of 0.938 and 0.548 for training and testing data. With the training score being very good but the testing score being poor, we can surmize this model is overfit with a high variance.

Comparing the Linear Regression to the Random Forest models, we can see that the lin reg model had a very small advantage on test data R2 scores (0.567 vs 0.548).

Aside from this, the Random Forest model did have a much higher R2 score when looking at the test data (0.938 vs 0.561), so maybe it can be tuned to be more accurate on the test data.



---


**Tuned Random Forest Model**

In [ ]:
# Find param candidates for tuning
rf_pipe.get_params()

In [ ]:
# Define param grid with options to try
params =  {'randomforestregressor__max_depth': [None,10,15,20],
          'randomforestregressor__n_estimators':[10,100,150,200],
          'randomforestregressor__min_samples_leaf':[2,3,4],
          }
# Instantiate the gridsearch
gridsearch = GridSearchCV(rf_pipe, params, cv=3, n_jobs=-1, verbose=1)

In [ ]:
# Fit the gridsearch on training data
gridsearch.fit(X_train, y_train)

Fitting 3 folds for each of 48 candidates, totalling 144 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('numeric',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         Index(['Item_Weight', 'Item_Visibility', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Location_Type'],
      dtype='object')...
                                                                                                        sparse_output=False))]),
                                                                         Index(['Item_Fat_Content', 'Item_Type', 'Outlet_Type'], dtype='object'))],
                                                          verbose_feature_names_out=False)),
                                       ('randomforestregressor',
                                        RandomForestRegressor())]),
             n_jobs=-1,
             param_grid={'randomforestregressor__max_depth': [None, 10, 15, 20],
                         'randomforestregressor__min_samples_leaf': [2, 3, 4],
                         'randomforestregressor__n_estimators': [10, 100, 150,
                                                                 200]},
             verbose=1)

In [ ]:
# Obtain best parameters
gridsearch.best_params_

{'randomforestregressor__max_depth': 10,
 'randomforestregressor__min_samples_leaf': 4,
 'randomforestregressor__n_estimators': 200}

In [ ]:
# Define and refit best model on entire training set
best_rf = gridsearch.best_estimator_
evaluate_regression(best_rf, X_train, y_train, X_test, y_test)

------------------------------------------------------------
Regression Metrics: Training Data
------------------------------------------------------------
- MAE = 660.325
- MSE = 886,435.845
- RMSE = 941.507
- R^2 = 0.700

------------------------------------------------------------
Regression Metrics: Test Data
------------------------------------------------------------
- MAE = 736.390
- MSE = 1,119,683.153
- RMSE = 1,058.151
- R^2 = 0.594


Tuning the parameters of max_depth, min_samples_leaf, and n_estimators had the effect of bringing our R2 value up from 0.548 in our default model to 0.594 in our tuned model.

# Evaluation

**Analyzing the scores**

The tuned random forest model is performing the best. It has the highest test R2 value at 0.594, and an R2 score of 0.700 for the training data, which are not far apart and indicates there is not strong overfitting. This model also has the lowest figure for RMSE, coming in at $1,058,151.

The default random forest on the other hand has training and testing scores of 0.938 and 0.548, showing it was overfit on the training data. It also has higher values for all mean errors.

The linear regression model has scores that are low for both training and testing data, 0.561 and 0.567 respectively. This shows the model has poor predictive ability and is underfit. Predictably, it had the highest RMSE at $1.092,937.

**Best Model**

The Random Forest model is our best performing model at the moment. It's R2 scores indicate it is the model with the best fit to our dataset, and it's RMSE is the lowest. This metric gives us a clearer picture than un-squared error metrics, as prices above and below the mean can cancel each other and make our error appear smaller.

However, in it's current state, the model does not have strong predictive power for the stakeholder.